In [21]:
import pandas as pd

In [22]:
df_accounts = pd.read_csv('data/accounts.csv')
df_alliance_ff = pd.read_csv('data/alliance_ff.csv')
df_alliance_membership = pd.read_csv('data/alliance_membership.csv')
df_chat_messages_1 = pd.read_csv('data/chat_messages_1.csv', dtype='unicode')
df_chat_messages_2 = pd.read_csv('data/chat_messages_2.csv', dtype='unicode')

Clean dataset chat_messages_1   

Drop unnecessary columns

In [23]:
columns_to_drop = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_1.drop(columns_to_drop, inplace=True, axis=1)
df_chat_messages_1.to_csv("modified_chat_messages_1.csv", index=False)

Clean dataset chat_messages_2

Drop unnecessary columns

In [ ]:
columns_to_drop2 = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_2.drop(columns_to_drop2, inplace=True, axis=1)
df_chat_messages_2.to_csv("modified_chat_messages_2.csv", index=False)

Concat two csv

In [ ]:
df_modified1 = pd.read_csv("modified_data/modified_chat_messages_1.csv", dtype='unicode')
df_modified2 = pd.read_csv("modified_data/modified_chat_messages_2.csv", dtype='unicode')

combined_df = pd.concat([df_modified1, df_modified2], ignore_index=True)
combined_df.to_csv("merged_message_data.csv", index=False)

Add offensive column

In [ ]:
merged_df = pd.read_csv("merged_message_data.csv", dtype='unicode')
merged_df['GENERAL_RISK'] = pd.to_numeric(merged_df['GENERAL_RISK'], errors='coerce')
merged_df['offensive'] = (merged_df['GENERAL_RISK'] >= 5).astype(int)
merged_df.to_csv("offensive_added.csv", index=False)

Concat two csv

In [ ]:
df_modified1 = pd.read_csv("modified_data/modified_chat_messages_1.csv", dtype='unicode')
df_modified2 = pd.read_csv("modified_data/modified_chat_messages_2.csv", dtype='unicode')

combined_df = pd.concat([df_modified1, df_modified2], ignore_index=True)
combined_df.to_csv("merged_message_data.csv", index=False)

Load Model

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report
model_name = "cardiffnlp/twitter-roberta-base-offensive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Train Model

In [41]:
from sklearn.model_selection import train_test_split
from torch import nn
import torch.optim as optim  # Import optimizer class

# Load your dataset
data = pd.read_csv("offensive_added.csv", dtype='unicode')

# Prepare text data for the model
cleaned_data = data[data["raw_message"].notna()]
filtered_data = cleaned_data[cleaned_data["raw_message"].str.isalpha()]
text = [item for sublist in filtered_data["raw_message"].tolist() for item in sublist]
encoded_data = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Freeze the pre-trained model layers (optional)
for param in model.base_model.parameters():
    param.requires_grad = False  # Freeze pre-trained weights

# Define optimizer and loss function
optimizer = optim.AdamW(model.classifier.parameters(), lr=2e-5)  # Adjust learning rate as needed
loss_fn = nn.CrossEntropyLoss()  # Cross-entropy loss for binary classification

# Define batch_size before the training loop
batch_size = 32  # Adjust this value as needed

# Training loop
epochs = 3  # Adjust number of epochs
for epoch in range(epochs):
    # Shuffle data for each epoch (optional)
    data = data.sample(frac=1).reset_index(drop=True)
    cleaned_data = data[data["raw_message"].notna()]
    filtered_data = cleaned_data[cleaned_data["raw_message"].str.isalpha()]
    text = [item for sublist in filtered_data["raw_message"].tolist() for item in sublist]
    encoded_data = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Training loop per batch
    for batch_idx in range(len(encoded_data) // batch_size):
        input_ids = encoded_data['input_ids'][batch_idx*batch_size:(batch_idx+1)*batch_size]
        attention_mask = encoded_data['attention_mask'][batch_idx*batch_size:(batch_idx+1)*batch_size]
        labels = torch.tensor(data["offensive"][batch_idx*batch_size:(batch_idx+1)*batch_size])  # Assuming "label" is your binary classification column

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Calculate loss
        loss = loss_fn(logits, labels)

        # Backward pass and parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress (optional)
        if batch_idx % 100 == 0:
            print(f"Epoch: {epoch+1}/{epochs}, Batch: {batch_idx+1}/{len(encoded_data) // batch_size}, Loss: {loss.item():.4f}")

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(encoded_data['input_ids'], data["offensive"], test_size=0.2, random_state=42)

# Training loop with validation
epochs = 3  # Adjust number of epochs
best_val_f1 = 0.0  # Initialize best validation F1-score
early_stopping_count = 0  # Counter for early stopping
for epoch in range(epochs):
    # ... your existing training loop per batch

    # Evaluate on validation set after each epoch
    with torch.no_grad():
        val_outputs = model(X_val, attention_mask=X_val['attention_mask'])
        val_logits = val_outputs.logits
        val_loss = loss_fn(val_logits, y_val)
        val_predictions = torch.argmax(val_logits, dim=1)

        # Calculate F1-score (or other metrics)
        from sklearn.metrics import f1_score
        val_f1 = f1_score(y_val, val_predictions)

        print(f"Epoch: {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val F1: {val_f1:.4f}")

        # Early stopping (optional)
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            early_stopping_count = 0  # Reset counter
            # Optionally, save the best model here
        else:
            early_stopping_count += 1
            if early_stopping_count >= 5:  # Adjust patience as needed
                print("Early stopping triggered!")
                break

from sklearn.metrics import classification_report

# After training, use classification report
y_pred = torch.argmax(model(X_test['input_ids'], attention_mask=X_test['attention_mask']).logits, dim=1)
print(classification_report(y_test, y_pred))

# Save the fine-tuned model (optional)
model.save_pretrained("your_fine_tuned_model")
